In [8]:
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np

createDatasets = False

In [9]:
def getTags(xml_path=None, xml_string=None, sentence_tag="sentence"):
    tree = None
    if xml_path:
        tree = ET.parse(xml_path)
    else:
        tree = ET.ElementTree(ET.fromstring(xml_string))
    root = tree.getroot()
    df = pd.DataFrame(columns=["orth", "base", "ctag", "sentence"])
    i = 0
    if sentence_tag == "sentence":
        for sentence in root.iter('sentence'):
            df = pd.concat([df, getSentenceTags(sentence, i)], ignore_index=True)
            i += 1
    elif sentence_tag == "chunk":
        for chunk in root.iter('chunk'):
            if chunk.attrib["type"] == "s":
                df = pd.concat([df, getSentenceTags(chunk, i)], ignore_index=True)
                i += 1
    else:
        raise ValueError("Wrong tag or not implemented analysis")
    return df

def getSentenceTags(sentence, s_idx):
    sentence_df = pd.DataFrame(columns=["orth", "base", "ctag", "sentence"])
    for tok in sentence.iter('tok'):
        orth = tok.find('orth').text
        for lex in tok.iter('lex'):
            if "disamb" in lex.attrib and lex.attrib['disamb']=="1":
                base = lex.find('base').text
                ctag = lex.find('ctag').text
                record = pd.DataFrame(data={"orth": [orth], "base": [base], "ctag": [ctag], "sentence": s_idx})
                sentence_df = pd.concat([sentence_df, record], ignore_index=True)
    return sentence_df

In [10]:
if createDatasets:
    clarin_df = getTags(xml_path="dataset/clarin/clarin-task-c.xml", sentence_tag="sentence")
    clarin_df.to_csv("tagers_results/claring.csv")

In [11]:
clarin_df = pd.read_csv("tagers_results/claring.csv")

In [14]:
n_sentences = len(set(clarin_df["sentence"]))
print("Liczba sentencji: " + str(n_sentences))
clarin_df.head(10)

Liczba sentencji: 1663


,Unnamed: 0,orth,base,ctag,sentence
0,0,Moje,moja,subst:pl:nom:f,0
1,1,niefortunne,niefortunny,adj:sg:nom:n:pos,0
2,2,pudła,pudło,subst:sg:gen:n,0
3,3,przygnębiły,przygnębić,praet:pl:m2:perf,0
4,4,mnie,ja,ppron12:sg:gen:m1:pri:akc,0
5,5,do,do,prep:gen,0
6,6,reszty,reszta,subst:sg:gen:f,0
7,7,.,.,interp,0
8,8,Zawsze,zawsze,adv:pos,1
9,9,miałem,miał,subst:sg:inst:m3,1


In [16]:
if createDatasets:
    gold_df = getTags(xml_path="dataset/PolEval/gold-task-c.xml", sentence_tag="chunk")
    gold_df.to_csv("tagers_results/gold.csv")

In [17]:
gold_df = pd.read_csv("tagers_results/gold.csv")

In [18]:
n_sentences = len(set(gold_df["sentence"]))
print("Liczba sentencji: " + str(n_sentences))
gold_df.head(10)

Liczba sentencji: 1671


,Unnamed: 0,orth,base,ctag,sentence
0,0,Moje,mój,adj:pl:nom:n:pos,0
1,1,niefortunne,niefortunny,adj:pl:nom:n:pos,0
2,2,pudła,pudło,subst:pl:nom:n,0
3,3,przygnębiły,przygnębić,praet:pl:n:perf,0
4,4,mnie,ja,ppron12:sg:acc:m1:pri:akc,0
5,5,do,do,prep:gen,0
6,6,reszty,reszta,subst:sg:gen:f,0
7,7,.,.,interp,0
8,8,Zawsze,zawsze,adv,1
9,9,miał,mieć,praet:sg:m1:imperf,1


In [21]:
def checkResult(pred_df, true_df, skip_ctag=False):
    correct = []
    incorrect = []
    bad_sentence_len = []
    for s_idx in set(pred_df["sentence"]):
        true_s = true_df[true_df["sentence"] == s_idx]
        pred_s = pred_df[pred_df["sentence"] == s_idx]
        if skip_ctag:
            true_s = true_s.drop(['ctag'], axis=1)
            pred_s = pred_s.drop(['ctag'], axis=1)
        t_s_len = len(true_s)
        p_s_len = len(pred_s)
        cor = []
        incor = []
        shift = 0
        if len(pred_s) != len(true_s):
            bad_sentence_len.append(s_idx)
        for i in range(p_s_len):
            ts_s_i = i + shift
            if ts_s_i < t_s_len and\
                    pred_s.iloc[i].equals(true_s.iloc[ts_s_i]):
                cor.append(pred_s.index.tolist()[i])
            elif ts_s_i + 1 < t_s_len and\
                    pred_s.iloc[i].equals(true_s.iloc[ts_s_i + 1]):
                cor.append(pred_s.index.tolist()[i])
                shift += 1
            elif 0 <= ts_s_i - 1 < t_s_len and\
                    pred_s.iloc[i].equals(true_s.iloc[ts_s_i - 1]):
                cor.append(pred_s.index.tolist()[i])
                shift -= 1
            else:
                incor.append(pred_s.index.tolist()[i])
        correct.append(cor)
        incorrect.append(incor)
    return correct, incorrect, bad_sentence_len

def countNotNone(listOfLists):
    list_len = 0
    for list_ in listOfLists:
        if len(list_) != 0:
            list_len += 1
    return list_len
def printComparisonResult(correct, incorrect, bad_sentence_len):
    print("Correct: " + str(countNotNone(correct)))
    print("Incorrect: " + str(countNotNone(incorrect)))
    print("Bad lenght sentences: " + str(len(bad_sentence_len)))

In [22]:
correct, incorrect, bad_sentence_len = checkResult(clarin_df, gold_df, skip_ctag=True)

In [23]:
printComparisonResult(correct, incorrect, bad_sentence_len)

Correct: 7
Incorrect: 1663
Bad lenght sentences: 1560


Sprawdzenie sentencji na pozycjach, w których wystąpiło dużo błędów

In [24]:
incorrect[:10]

[[0],
 [9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38],
 [39, 40, 41, 42, 43, 44, 45, 46, 47, 50],
 [63, 66, 72],
 [77, 87, 94, 95, 96, 97, 98, 99],
 [100, 101, 102, 103, 104, 105, 106, 107, 108],
 [109, 110, 111, 112, 113, 114, 115, 116, 117, 118],
 [119, 120, 121, 122, 123, 124, 125],
 [126, 127, 128, 129, 130, 131, 132],
 [133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145]]

### Zdesynchonizowane sentencje
Z poniższych danych wynika, że tagery różnie rozumieją sentencję.  
Nie jesteśmy w stanie przez to zaimplementować bezpieczniejszej ewaluacji modeli - sentencja po sentencji.

In [25]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    return diff_df
idx = 7
dataframe_difference(clarin_df[clarin_df["sentence"]==idx], gold_df[gold_df["sentence"]==idx])

,Unnamed: 0,orth,base,ctag,sentence,_merge
0,119,Z,z,prep:gen:nwok,7,left_only
1,120,czego,czego,adv:pos,7,left_only
2,121,strzelał,strzelać,praet:sg:m1:imperf,7,left_only
3,122,eś,być,aglt:sg:sec:imperf:wok,7,left_only
4,123,przed,przed,prep:acc:nwok,7,left_only
5,124,wojną,wojna,subst:sg:inst:f,7,left_only
6,125,?,?,interp,7,left_only
7,109,-,-,interp,7,right_only
8,110,Chyba,chyba,qub,7,right_only
9,111,był,być,praet:sg:m1:imperf,7,right_only


# Wersja bez sentencji

In [28]:
def checkResult(pred_df, true_df, skip_ctag=False, n_neigh=2):
    comparison = pd.DataFrame(columns=["pred_idx", "true_idx"])
    shift = 0
    list_true_idx = -1
    if skip_ctag:
        pred_df = pred_df.drop(['ctag'], axis=1)
        true_df = true_df.drop(['ctag'], axis=1)
    for i in range(len(pred_df)):
        true_i, shift = checkNeighbours(pred_df, true_df, n_neigh, i, shift, list_true_idx)
        if true_i is not None:
            record = pd.DataFrame(data={"pred_idx": [i], "true_idx": [true_i]})
            comparison = pd.concat([comparison, record], ignore_index=True)
            list_true_idx = true_i
    return comparison

def checkNeighbours(pred_df, true_df, n_neigh, i, shift, last_idx):
    t_len = len(true_df)
    t_index = i + shift
    for neighbour in range(-n_neigh, n_neigh + 1):
        if 0 < t_index + neighbour < t_len and\
                pred_df.iloc[i].equals(true_df.iloc[t_index + neighbour]) and\
                t_index + neighbour > last_idx:
            return t_index + neighbour, shift + neighbour
    return None, shift

def countNotNone(array):
    correct = 0
    incorrect = 0
    for cell in array:
        if cell is not None:
            correct += 1
        else:
            incorrect += 1
    print("Correct: " + str(correct))
    print("Incorrect: " + str(incorrect))

## 1. Synchronizacja wyników tagowania
W wyniku tagowania niektóre tokeny mogły zostać rozczłonkowane na mniejsze (np. mogłem -> móc + em (aglutenat)). Inne mogą traktować takie słowo jako jeden tag. Taka różnica traktowana jest jako błąd. Przez takie zjawisko nie możemy porównywać tag po tagu, następne tagi mogą pojawiać się na pozycjach wcześniejszych lub późniejszych w stosunku do bazowego tagsetu. Aby zaradzić przesunięciom w pierwszej kolejności dokonywana jest analiza na jakich pozycjach występują odpowiadające sobie słowa. 

In [29]:
clarin_orto = clarin_df[["orth"]]
gold_orto = gold_df[["orth"]]
correct_orto = checkResult(clarin_orto, gold_orto, n_neigh=2)
correct_orto

,pred_idx,true_idx
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
2165,27418,27480
2166,27419,27481
2167,27442,27506
2168,27460,27525


### Mała liczba wyników
Ponieważ liczba zestawionych prawidłowo tagów jest mała (niecałe 10%),  
warto przy analizach zmieniać współczynnik n_neigh - jest on odpowiedzialny  
za patrzenie N sąsiadujących tagów w przód i tył w stosunku do pozycji, na  
której oczekiwaliśmy tagu. Warto jednak mieć na uwadze, że duże wartości  
mogą spowodować sięganie po identyczne słowa, które wystąpiły wcześniej,  
zupełnie nie związane z tagiem bazowym na badanej pozycji.

In [34]:
print(clarin_df.iloc[20:30])
gold_df.iloc[20:30]

    Unnamed: 0       orth       base                    ctag  sentence
20          20        nie        nie                    conj         1
21          21  rozstawał  rozstawać      praet:sg:m1:imperf         1
22          22         em        być  aglt:sg:pri:imperf:wok         1
23          23        się        się                     qub         1
24          24         ze          z            prep:gen:wok         1
25          25   strzelbą   strzelba         subst:sg:inst:f         1
26          26          ,          ,                  interp         1
27          27          a          a                    conj         1
28          28         tu         tu                 adv:pos         1
29          29      wśród      wśród                prep:gen         1


,Unnamed: 0,orth,base,ctag,sentence
20,20,dzieciństwa,dzieciństwo,subst:sg:gen:n,1
21,21,nie,nie,qub,1
22,22,rozstawał,rozstawać,praet:sg:m1:imperf,1
23,23,em,być,aglt:sg:pri:imperf:wok,1
24,24,się,się,qub,1
25,25,ze,z,prep:inst:wok,1
26,26,strzelbą,strzelba,subst:sg:inst:f,1
27,27,",",",",interp,1
28,28,a,a,conj,1
29,29,tu,tu,adv,1


### Ctag - analiza morfosyntaktyczna
Kolejnym problemem, który można napotkać podczas porównania wyników  
dwóch różnych tagerów jest sposób oznaczania wyniku analizy  
morfosyntaktycznej. Przykład: słowo "tu" clarin opisał jako  
"adv:pos"(28), gold standard zaanotował je jako "adv"(29).  
Z tego powodu zdecydowano się na dwie osobne analizy:
- bez ctag,
- z uwzględnieniem ctag.

In [30]:
# Analiza bez ctag
pred_df = clarin_df.loc[clarin_df.index.isin(correct_orto["pred_idx"]), ["orth", "base"]]
true_df = gold_df.loc[gold_df.index.isin(correct_orto["true_idx"]), ["orth", "base"]]

comparison = checkResult(pred_df, true_df, n_neigh=0)
comparison

,pred_idx,true_idx
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
2092,2165,2165
2093,2166,2166
2094,2167,2167
2095,2168,2168


In [35]:
# Analiza z ctag
pred_ctag_df = clarin_df.loc[clarin_df.index.isin(correct_orto["pred_idx"]), ["orth", "base", "ctag"]]
true_ctag_df = gold_df.loc[gold_df.index.isin(correct_orto["true_idx"]), ["orth", "base", "ctag"]]

comparison_ctag = checkResult(pred_ctag_df, true_ctag_df, n_neigh=0)
comparison_ctag

,pred_idx,true_idx
0,4,4
1,5,5
2,6,6
3,9,9
4,10,10
...,...,...
1735,2165,2165
1736,2166,2166
1737,2167,2167
1738,2168,2168


In [61]:
def compareTagerResults(tager_df, gold_df):
    tager_orto = tager_df[["orth"]]
    gold_orto = gold_df[["orth"]]
    correct_orto = checkResult(tager_orto, gold_orto, n_neigh=2)
    
    pred_df = tager_df.loc[tager_df.index.isin(correct_orto["pred_idx"]), ["orth", "base"]]
    true_df = gold_df.loc[gold_df.index.isin(correct_orto["true_idx"]), ["orth", "base"]]
    comparison = checkResult(pred_df, true_df, n_neigh=0)
    
    pred_ctag_df = clarin_df.loc[clarin_df.index.isin(correct_orto["pred_idx"]), ["orth", "base", "ctag"]]
    true_ctag_df = gold_df.loc[gold_df.index.isin(correct_orto["true_idx"]), ["orth", "base", "ctag"]]
    comparison_ctag = checkResult(pred_ctag_df, true_ctag_df, n_neigh=0)
    printTagerResults(tager_df, comparison, comparison_ctag, correct_orto)
    return comparison, comparison_ctag, correct_orto

def printTagerResults(df, comparison, comparison_ctag, correct_orto):
    print("Liczba tagów: " + str(df.shape[0]))
    print("Liczba tagów zsynchonizowanych: " + str(correct_orto.shape[0]))
    print("Bez ctag - {0:.2f} %".format(comparison.shape[0]/correct_orto.shape[0]))
    print("Z ctag   - {0:.2f} %".format(comparison_ctag.shape[0]/correct_orto.shape[0]))

# Wyniki

In [57]:
"Liczba tagów w golden standard: " + str(gold_df.shape[0])

'Liczba tagów w golden standard: 27563'

## Clarin

In [62]:
clarin_df = getTags(xml_path="dataset/clarin/clarin-task-c.xml")
comparison_clarin, comparison_ctag_clarin, correct_orto_clarin = compareTagerResults(clarin_df, gold_df)

Liczba tagów: 29346
Liczba tagów zsynchonizowanych: 2170
Bez ctag - 0.97 %
Z ctag   - 0.80 %


(     pred_idx true_idx
 0           1        1
 1           2        2
 2           3        3
 3           4        4
 4           5        5
 ...       ...      ...
 2092     2165     2165
 2093     2166     2166
 2094     2167     2167
 2095     2168     2168
 2096     2169     2169
 
 [2097 rows x 2 columns],
      pred_idx true_idx
 0           4        4
 1           5        5
 2           6        6
 3           9        9
 4          10       10
 ...       ...      ...
 1735     2165     2165
 1736     2166     2166
 1737     2167     2167
 1738     2168     2168
 1739     2169     2169
 
 [1740 rows x 2 columns],
      pred_idx true_idx
 0           1        1
 1           2        2
 2           3        3
 3           4        4
 4           5        5
 ...       ...      ...
 2165    27418    27480
 2166    27419    27481
 2167    27442    27506
 2168    27460    27525
 2169    27469    27534
 
 [2170 rows x 2 columns])

## KRNNT

In [63]:
krnnt_df = pd.read_csv("./dataset/krnnt/test-raw.csv")
comparison_krnnt, comparison_ctag_krnnt, correct_orto_krnnt = compareTagerResults(krnnt_df, gold_df)

Liczba tagów: 27577
Liczba tagów zsynchonizowanych: 22499
Bez ctag - 0.98 %
Z ctag   - 0.01 %


## MorphoDiTa

In [ ]:
morphoDiTa_df = getTags(xml_path="/home/stachu/PycharmProjects/sem2/NLP/NLP1/dataset/morphoDiTa/test-raw.xml")
comparison_morphoDiTa, comparison_ctag_morphoDiTa, correct_orto_morphoDiTa = compareTagerResults(morphoDiTa_df, gold_df)